In [1]:
import tensorflow as tf
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Input, Conv2D, Flatten,  Softmax, BatchNormalization, MaxPooling2D, ELU, Dense, ReLU, Dropout
from keras.layers import Add, GlobalAveragePooling2D
from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from keras import regularizers


In [2]:
tf.__version__

'2.10.0'

# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`.

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [3]:
df = open("C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/dan_train.csv").read().splitlines()
games = [i.split(',',2)[-1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [2]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0

Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [3]:
def extract_features(moves):
    features = np.zeros((19, 19, 11))

    for i, move in enumerate(moves):
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]

        features[row, column, 0] = 1 if color == 'W' else -1  # 表示下棋的顏色
        features[row, column, 1] = 1 if i % 2 == 0 else -1  # 表示輪到哪一方下棋
        features[row, column, 2] = 1  # 表示該位置上是否已下棋

        # 最後一步的信息
        features[row, column, 3] = 1 if i == len(moves) - 1 else 0

        # 最近幾步的分佈情況
        for j in range(1, min(5, i + 1)):
            prev_move = moves[i - j]
            prev_column = coordinates[prev_move[2]]
            prev_row = coordinates[prev_move[3]]
            features[prev_row, prev_column, j] = 1 if prev_move[0] == color else -1
        
    # 其他特徵的計算...
    # 這裡可以添加更多特徵的計算邏輯，例如區域控制、連子信息等

    return features


def prepare_input(moves):
    x = np.zeros((19, 19, 11))

    # 在這裡調用特徵提取函數
    features = extract_features(moves)

    # 將提取的特徵放入輸入張量的特定通道
    x[:, :, 0:5] = features[:, :, 0:5]

    # 其他通道的處理...
    # 例如，你可以在不同的通道上添加其他特徵

    # 將你原本的邏輯放在這裡...

        

    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[last_move_row, last_move_column, 3] = 1

    x[:, :, 2] = np.where(x[:, :, 2] == 0, 1, 0)

    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [6]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 100160, Total Moves: 22853380


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [7]:
import random


x = []
y = []
games = random.sample(games, 2000)
for game in games:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

In [8]:
x.shape

(456269, 19, 19, 11)

In [9]:
y.shape

(456269,)

In [10]:
y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [11]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

In [ ]:
from keras.models import Sequential
from keras.layers import Average

def create_model1():
    inputs = Input(shape=(19, 19, 11))
    outputs = Conv2D(kernel_size=12, filters=64, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=16, filters=32, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=16, filters=32, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=16, filters=32, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=16, filters=1, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)

    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    outputs = Dropout(0.05)(outputs)

    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


# Create an ensemble model using Average and Lambda layers
models = [create_model1() for i in range(5)]
model_input = Input(shape=(19, 19, 11))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)
ensemble_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Training

### Simple DCNN Model:

In [ ]:
def create_model():
    inputs = Input(shape=(19, 19, 11))

    x = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(inputs)
    x = BatchNormalization()(x)

    x = Conv2D(kernel_size=3, filters=128, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)

    x = Conv2D(kernel_size=3, filters=256, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)

    
    x = Conv2D(kernel_size=3, filters=512, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(361, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    outputs = Dense(361, activation='softmax')(x)

    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.0005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [12]:
def create_model6():
    inputs = Input(shape=(19, 19, 11))
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(inputs)
    outputs = BatchNormalization()(outputs)
    
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=16, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
   
    outputs = Conv2D(kernel_size=16, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
   
    outputs = Conv2D(kernel_size=16, filters=1, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)

    outputs = ELU()(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    
    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def create_model7():
    inputs = Input(shape=(19, 19, 11))

    x = Conv2D(kernel_size=3, filters=64, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=3, filters=128, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=3, filters=256, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    outputs = Dense(361, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.0005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
from keras.models import Sequential
def create_model5():
    model = Sequential()

    model.add(Conv2D(kernel_size=3, filters=64, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.003), input_shape=(19, 19, 11)))
    model.add(BatchNormalization())

    model.add(Conv2D(kernel_size=3, filters=128, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.003)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv2D(kernel_size=3, filters=256, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.003)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv2D(kernel_size=3, filters=512, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.003)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(361, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(361, activation='softmax'))

    opt = Adam(learning_rate=0.0005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [14]:
model6 = create_model6()
model6.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 11)]      0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 64)        34560     
                                                                 
 batch_normalization (BatchN  (None, 19, 19, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 64)        102464    
                                                                 
 batch_normalization_1 (Batc  (None, 19, 19, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 32)        51232 

In [ ]:
ensemble_model.summary()

In [4]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
    
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPU


In [ ]:
history = ensemble_model.fit(
    x = x_train,
    y = y_train,
    batch_size = 256,
    epochs = 20,
    validation_data=(x_val, y_val),
)

In [15]:
def data_generator(x, y, batch_size):
    while True:
        for batch_start in range(0, len(x), batch_size):
            batch_end = batch_start + batch_size
            x_batch = x[batch_start:batch_end]
            y_batch = y[batch_start:batch_end]
            yield x_batch, y_batch

# 創建生成器
batch_size = 512  # 設置合適的批次大小
train_generator = data_generator(x_train, y_train, batch_size)
val_generator = data_generator(x_val, y_val, batch_size)

history = model6.fit(
    train_generator,
    steps_per_epoch=len(x_train)//batch_size,
    epochs=40,
    validation_data=val_generator,
    validation_steps=len(x_val)//batch_size
)

Epoch 1/40
802/802 [==============================] - 144s 147ms/step - loss: 4.8243 - accuracy: 0.1047 - val_loss: 4.1346 - val_accuracy: 0.2473
Epoch 2/40
802/802 [==============================] - 116s 145ms/step - loss: 3.5493 - accuracy: 0.3283 - val_loss: 3.5237 - val_accuracy: 0.3466
Epoch 3/40
802/802 [==============================] - 118s 147ms/step - loss: 3.2054 - accuracy: 0.3709 - val_loss: 3.2639 - val_accuracy: 0.3760
Epoch 4/40
802/802 [==============================] - 114s 143ms/step - loss: 3.0698 - accuracy: 0.3844 - val_loss: 3.1176 - val_accuracy: 0.3787
Epoch 5/40
802/802 [==============================] - 114s 143ms/step - loss: 2.9978 - accuracy: 0.3913 - val_loss: 3.1384 - val_accuracy: 0.3619
Epoch 6/40
802/802 [==============================] - 114s 143ms/step - loss: 2.9484 - accuracy: 0.3959 - val_loss: 3.0215 - val_accuracy: 0.3836
Epoch 7/40
802/802 [==============================] - 115s 143ms/step - loss: 2.9116 - accuracy: 0.3992 - val_loss: 3.0742 -

In [16]:
model6.save('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial7.h5')

In [ ]:
def data_generator(x, y, batch_size):
    while True:
        for batch_start in range(0, len(x), batch_size):
            batch_end = batch_start + batch_size
            x_batch = x[batch_start:batch_end]
            y_batch = y[batch_start:batch_end]
            yield x_batch, y_batch

# 創建生成器
batch_size = 256  # 設置合適的批次大小
train_generator = data_generator(x_train, y_train, batch_size)
val_generator = data_generator(x_val, y_val, batch_size)

history2 = model2.fit(
    train_generator,
    steps_per_epoch=len(x_train)//batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(x_val)//batch_size
)

In [ ]:
model2.save('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial2.h5')

In [ ]:
def data_generator(x, y, batch_size):
    while True:
        for batch_start in range(0, len(x), batch_size):
            batch_end = batch_start + batch_size
            x_batch = x[batch_start:batch_end]
            y_batch = y[batch_start:batch_end]
            yield x_batch, y_batch

# 創建生成器
batch_size = 256  # 設置合適的批次大小
train_generator = data_generator(x_train, y_train, batch_size)
val_generator = data_generator(x_val, y_val, batch_size)

history3 = model3.fit(
    train_generator,
    steps_per_epoch=len(x_train)//batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(x_val)//batch_size
)

In [ ]:
model3.save('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial3.h5')

In [18]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

# 載入模型
loaded_model_1 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial.h5')
loaded_model_2 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial2.h5')
loaded_model_3 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial3.h5')
loaded_model_4 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial4.h5')
loaded_model_5 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial5.h5')
loaded_model_6 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial6.h5')
loaded_model_7 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial7.h5')

# 使用載入的模型進行預測
pred_probs_model1 = loaded_model_1.predict(x_val)
pred_probs_model2 = loaded_model_2.predict(x_val)
pred_probs_model3 = loaded_model_3.predict(x_val)
pred_probs_model4 = loaded_model_4.predict(x_val)
pred_probs_model5 = loaded_model_5.predict(x_val)
pred_probs_model6 = loaded_model_6.predict(x_val)
pred_probs_model7 = loaded_model_7.predict(x_val)

# 投票法 Ensemble
def ensemble_voting(pred_probs_list):
    final_predictions = []
    for i in range(len(pred_probs_list[0])):
        votes = [pred_probs[i].argmax() for pred_probs in pred_probs_list]
        #votes = [pred_probs[i] for pred_probs in pred_probs_list]
        final_predictions.append([max(set(votes), key=votes.count)])
        #average_vote = np.mean(votes, axis=0)
        #final_predictions.append([votes])
      
    return final_predictions

# 將模型預測機率組成的列表傳入投票法
final_predictions = ensemble_voting([pred_probs_model1, pred_probs_model2, pred_probs_model3, pred_probs_model4, pred_probs_model5, pred_probs_model6,pred_probs_model7])

# 將 y_val 轉換為單一標籤形式
y_val_single_label = y_val.argmax(axis=1)

# 計算 Ensemble 後的準確率
accuracy = accuracy_score(y_val_single_label, final_predictions)
print("Ensemble Accuracy:", accuracy)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

# Testing

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
```

- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

The code block below is to use **dan_test_public.csv** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [5]:
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

In [6]:
from keras.models import load_model

df = open('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]


# 投票法 Ensemble
def ensemble_voting(pred_probs_list):
    final_predictions = []
    for i in range(len(pred_probs_list[0])):
        #votes = [pred_probs[i].argmax() for pred_probs in pred_probs_list]
        votes = [pred_probs[i] for pred_probs in pred_probs_list]
        #final_predictions.append([max(set(votes), key=votes.count)])
        average_vote = np.mean(votes, axis=0)
        final_predictions.append(average_vote)
    return final_predictions

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)

# 載入模型
loaded_model_1 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial.h5')
loaded_model_2 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial2.h5')
loaded_model_3 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial3.h5')
loaded_model_4 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial4.h5')
loaded_model_5 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial5.h5')
loaded_model_6 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial6.h5')
loaded_model_7 = load_model('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial7.h5')

pred_probs_model1 = loaded_model_1.predict(x_testing)
pred_probs_model2 = loaded_model_2.predict(x_testing)
pred_probs_model3 = loaded_model_3.predict(x_testing)
pred_probs_model4 = loaded_model_4.predict(x_testing)
pred_probs_model5 = loaded_model_5.predict(x_testing)
pred_probs_model6 = loaded_model_6.predict(x_testing)
pred_probs_model7 = loaded_model_7.predict(x_testing)

predictions = ensemble_voting([pred_probs_model1, pred_probs_model2, pred_probs_model3, pred_probs_model4, pred_probs_model5, pred_probs_model6, pred_probs_model7])
prediction_chars = top_5_preds_with_chars(predictions)


# Save results to PublicUpload.csv
with open('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/PublicUpload1128-3.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

344/344 [==============================] - 2s 6ms/step


In [7]:
df = open('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/dan_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)

pred_probs_model1 = loaded_model_1.predict(x_testing)
pred_probs_model2 = loaded_model_2.predict(x_testing)
pred_probs_model3 = loaded_model_3.predict(x_testing)
pred_probs_model4 = loaded_model_4.predict(x_testing)
pred_probs_model5 = loaded_model_5.predict(x_testing)
pred_probs_model6 = loaded_model_6.predict(x_testing)
pred_probs_model7 = loaded_model_7.predict(x_testing)

predictions = final_predictions = ensemble_voting([pred_probs_model1, pred_probs_model2, pred_probs_model3, pred_probs_model4])
prediction_chars = top_5_preds_with_chars(predictions)


# Save results to PublicUpload.csv
with open('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/PublicUpload1128-3.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

344/344 [==============================] - 2s 6ms/step


# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!

In [ ]:
print(prediction_chars)

In [ ]:
df = open('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/PublicUpload-1127.csv').read().splitlines()
games = [i.split(',')[0] for i in df]
games